<a href="https://colab.research.google.com/github/laq6661/Monocular-camera-project/blob/master/3d_sk_de_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import imutils
import cv2
import os
from keras.layers import Dropout
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Convolution2D, ConvLSTM2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from google.colab import drive

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

data = np.load('/content/drive/My Drive/data_sk/sk_lstmdata_h2.npy')
print(np.shape(data))

(1000000, 3, 150)


In [4]:
labels=np.load('/content/drive/My Drive/data_sk/sk_lstmlabel_h2.npy')
print(labels[0:5])
print(np.shape(labels))

[32 32 32 32 32]
(1000000,)


In [0]:
data = np.array(data)
labels = np.array(labels)
# for a in range(1,53,1):
#   print("number of ",a,"data is:",list(labels).count(a))

In [6]:
le = LabelEncoder().fit(labels)   # LabelEncoder可以将标签分配一个0—n_classes-1之间的编码 
print(labels)
labels = np_utils.to_categorical(le.transform(labels), 51)  # 2是num_class表示输出的是2列数据的意思
print(np.shape(labels))

[32 32 32 ...  1  1  1]
(1000000, 51)


In [0]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0.15, 
                                                 stratify = labels, random_state = 42)

In [8]:
print(np.shape(data)[2])
print(trainX[0,:])
print(np.shape(trainY))
print(trainY)
print(np.shape(testX))
print(testY)

150
[[ 0.073673 -0.090314  3.247111  0.052616  0.195476  3.230546  0.031933
   0.475514  3.206363  0.022293  0.60432   3.219299 -0.068336  0.338712
   3.118202 -0.114874  0.123874  3.091527 -0.092114 -0.092095  3.060054
  -0.076253 -0.157682  3.063973  0.188879  0.371519  3.293297  0.318848
   0.233853  3.239884  0.408983  0.289406  3.109535  0.440857  0.333603
   3.048019  0.025247 -0.091219  3.180043 -0.046264 -0.427051  3.240552
  -0.046838 -0.743973  3.288657 -0.019736 -0.831222  3.221023  0.119465
  -0.087956  3.245849  0.067814 -0.418577  3.327128  0.018829 -0.735643
   3.344908  0.041292 -0.813701  3.264805  0.037075  0.406359  3.214335
  -0.069127 -0.220442  3.071297 -0.04328  -0.13829   3.058429  0.479459
   0.361938  2.997326  0.434137  0.368148  3.028286  0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.        0.        0.        0.        0.        0.        0.
   0.       

In [9]:
model = Sequential()
model.add(LSTM(150,input_shape=(np.shape(data)[1],np.shape(data)[2]),return_sequences = True))
model.add(Conv1D(kernel_size=3,filters=25,activation='relu',padding="same"))
model.add(MaxPooling1D(pool_size=3,strides=1,padding='same'))
model.add(Conv1D(kernel_size=3, filters=50,  activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(1024))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.2))
# model.add(Dense(1024))
# # #model.add(BatchNormalization())
# model.add(Activation('relu'))
model.add(Dense(51, activation='softmax'))
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 150)            180600    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 3, 25)             11275     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 25)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3, 50)             3800      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 2, 50)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100)               0         


In [14]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

H = model.fit(trainX, trainY, validation_data = (testX, testY),
              batch_size = 128, epochs = 12, verbose = 1)  #verbose = 1显示进度条

Train on 850000 samples, validate on 150000 samples
Epoch 1/12
850000/850000 [==============================] - 123s 145us/step - loss: 0.0170 - acc: 0.9941 - val_loss: 0.0151 - val_acc: 0.9948
Epoch 2/12
850000/850000 [==============================] - 122s 143us/step - loss: 0.0169 - acc: 0.9941 - val_loss: 0.0163 - val_acc: 0.9944
Epoch 3/12
850000/850000 [==============================] - 121s 142us/step - loss: 0.0168 - acc: 0.9942 - val_loss: 0.0160 - val_acc: 0.9945
Epoch 4/12
850000/850000 [==============================] - 120s 141us/step - loss: 0.0167 - acc: 0.9942 - val_loss: 0.0153 - val_acc: 0.9948
Epoch 5/12
850000/850000 [==============================] - 122s 144us/step - loss: 0.0165 - acc: 0.9943 - val_loss: 0.0142 - val_acc: 0.9951
Epoch 6/12
850000/850000 [==============================] - 121s 143us/step - loss: 0.0164 - acc: 0.9943 - val_loss: 0.0148 - val_acc: 0.9949
Epoch 7/12
850000/850000 [==============================] - 122s 144us/step - loss: 0.0163 - acc

In [15]:
predictions = model.predict(testX, batch_size = 128)
print(np.shape(predictions))
print(predictions.argmax(axis =1))
print(testY.argmax(axis = 1))
leclass = [str(i) for i in le.classes_]
print(leclass)
print(classification_report(testY.argmax(axis = 1), predictions.argmax(axis = 1),
                           target_names = leclass))

(150000, 51)
[39 29 43 ... 41 41 42]
[39 29 43 ... 41 41 42]
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51']
              precision    recall  f1-score   support

           1       0.82      0.84      0.83      1537
           2       0.89      0.86      0.87      5555
           3       0.87      0.87      0.87      4976
           4       0.86      0.88      0.87      2848
           5       0.96      0.87      0.91      2432
           6       0.83      0.76      0.80      2918
           7       0.93      0.90      0.91      2522
           8       0.86      0.89      0.87      4403
           9       0.91      0.87      0.89      1151
          10       0.93      0.93      0.93      5480
          11       0.97      0.89      0.93   

In [0]:
model.save("/content/drive/My Drive/data_sk/sk_lstm85.hdf5")

In [0]:
from keras.models import load_model
loaded_model = load_model("/content/drive/My Drive/data_sk/sk_lstm85.hdf5")